# Experiment with Azure Digital Tween Service
[Location](https://explorer.digitaltwins.azure.net/?tid=76a2ae5a-9f00-4f6b-95ed-5d33d77c4d61&eid=digitaltwins.api.weu.digitaltwins.azure.net)

## Set up

### Create Service Instance
Create instance as described [here](https://learn.microsoft.com/en-gb/azure/digital-twins/quickstart-azure-digital-twins-explorer)

### Define models
To initial define set of models (ontology) was used Office 365 copilot and [prompt](prompts/Define%20models.txt). Some extra tuning was required.
Models are stored in `ontology` folder.
Then models were uploaded to service.

### Ontology
The service define several models as follow:

- Model name: `CentralUnit` - which is main unit of the system exists in one instance. 
    If user want to create new CentralUnit, you must refuse it with message: "Central Unit has to be created manually!"

- Model name: `ControlHub` - which is connection unit between _Central Unit_ and _Termo Sensors_. 

- Model name: `TermoSensor` - which conects to _Control Hub_ and reports temperature value.

## Gen AI
Utilise GenAI to work with toplogy
Add, update, remove digital twin to the graph.

### Overview

![ReAct](diagrams/ReAct.png)


### Init

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
digital_twin_endpoint = os.getenv("AZURE_DIGITAL_TWINS_ENDPOINT")

import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

Init Semantic Kernel

In [2]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureChatPromptExecutionSettings
from semantic_kernel.kernel import Kernel

kernel = Kernel(log=logger)

service_id = "default"
kernel.add_service(
    AzureChatCompletion(service_id=service_id),
)

DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/home/vscode/.local/lib/python3.12/site-packages/certifi/cacert.pem'


### Plugin (Tools)

Define plugins that helps LLM to perform tasks on digital twins.

- `get_module_definition` - defines modules and rules of handling digital twins. It's implementation of RAG architecture where user's input _augmented_ with details of the model that needs to be maintained.
- `query_service` - requests information about _digital twin_ from Azure Digital Twins Query API.
- `update_twin`, `delete_twin` - creates or deletes _digital twin_.
- `update_relationship`, `delete_relationship` - creates or deletes _relationship_ between digital twins.


In [24]:
from typing import Annotated
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from azure.digitaltwins.core import DigitalTwinsClient
from azure.identity import DefaultAzureCredential
import json

credentials = DefaultAzureCredential()

class AdtServicePlugin:
    """
    Description: ADTServicePlugin provides a set of functions to manipulate and query digital twins.

    Usage:
        kernel.import_plugin_from_object(AdtServicePlugin(), plugin_name="adt_servcie")

    Examples:
        {{adt_service.get_model_definition}} => Gets DTDL model's description.
    """

    client = DigitalTwinsClient(digital_twin_endpoint, credential=credentials)

    @kernel_function(
            name="get_model_definition",
            description="Used to get the model definition in Digital Twin Definition Language (DTDL) format. Should be used to get model specification before operation on digital twin.")
    def get_module_definition(
        self,
        model_type: Annotated[str, "Model type that definition is required. For example: ControlHub or TermoSensor"]
    ) -> Annotated[str, "DTDL of the requested model and useful instruction how to use it.  If model name is not valid, the tool returns NA it means that execution is failed."]:
        """Gets the model definition in DTDL format by model name"""
        if model_type == "ControlHub":
            with open('ontology/ControlHub.json', 'r') as file:
                definition = file.read()

            instructions = """
            IMPORTANT NOTES:
            1. The model id in $metadata section is dtmi:com:example:ControlHub;1
            2. When requested creation new digital twin it must be performed in 2 steps:
                - create new digital twin of ControlHub model
                - create relationship between new digital twin and CentralUnit twin! ID of CentalUnit twin must be queried through Azure Digital Twins Query API.
                  $relationship Name should be "connects".  Source of the relationship is CentalUnit twin, target is ControlHub twin. Format of the parameter is specified in update_relationship function.
            """
            return definition + instructions
        elif model_type == "TermoSensor":
            with open('ontology/TermoSensor.json', 'r') as file:
                definition = file.read()

            instructions = """
            IMPORTANT NOTES:
            1. The model id in $metadata section is dtmi:com:example:TermoSensor;1
            2. When requested creation new digital twin it must be performed in 2 steps:
                - create new digital twin of TermoSensor model
                - create relationship between new digital twin and ControlHub twin. 
                    $relationship name should be "connects"
                    To make this relationship you must know id of ControlHub twin, this can be obtained by several ways:
                    - user can specify id of ControlHub twin in initial request - in this case you have this information.
                    - user may specidy name of ControlHub - in this case you must query its id through Azure Digital Twins Query API.
                    - if user does not provide neither name nor id you should use available for you human tool and ask the name of ControlHub twin. And then use it to get id as specified above.
            """
            return definition + instructions
        else:
            return "NA"

    @kernel_function(
        name="create_twin",
        description="Creates (adds) new digital twin in Azure Digital Twins Service."
    )
    def create_twin(
        self,
        twin_id: Annotated[str, "Unique identifier of the new digital twin. For example: ControlHub_1"],
        properties: Annotated[dict, """Properties of the new digital twin. The dictionary represent digital tween to be sent to the topology.
            The format should be in Azure Digital Twins Definition Language (DTDL).
            IMPORTANT! The metadata section should contain id of the model, for example if
            model id is dtmi:com:example:ControlHub;1 metadata section should looks like:
            "$metadata": {
                "$model": "dtmi:com:example:ControlHub;1"
            },

            The format must be valid to be sent as twin_data in following below example.
                
            from azure.digitaltwins.core import DigitalTwinsClient
            
            client = DigitalTwinsClient(digital_twin_endpoint, credential)
            client.upsert_digital_twin(twin_id, twin_data)
        """],
    ) -> Annotated[str, "Result of the operation. If operation is successful, the tool returns 'OK'. Otherwise, it returns 'FAIL' and the reason of failure, you're welcome to analyse error and fix it"]:
        try:
            self.client.upsert_digital_twin(twin_id, properties)
            return "OK"
        except Exception as e:
            print("Error creating digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
        
    @kernel_function(
        name="delete_twin",
        description="Deletes digital twin in Azure Digital Twins Service."
    )
    def delete_twin(
        self,
        twin_id: Annotated[str, """Unique identifier of the digital twin that is going to be deleted.
            The identifier is a value of $dtId requested from Azure Digital Twins Query API"""],
    ) -> Annotated[str, "Result of the operation. If operation is successful, the tool returns 'OK'. Otherwise, it returns 'FAIL' and the reason of failure, you're welcome to analyse error and fix it"]:
        try:
            self.client.delete_digital_twin(twin_id)
            return "OK"
        except Exception as e:
            print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
    
    @kernel_function(
        name="update_relationship",
        description="Creates or updates a relationship between two twins in Azure Digital Twins Service."
    )
    def update_relationship(
        self,
        twin_id: Annotated[str, """Unique identifier of the digital twin that is going to be the source of the connection.
            The identifier is a value of $dtId requested from Azure Digital Twins Query API"""],
        relationship_id: Annotated[str, "Unique identifier of the new relationship. Generate it if not specified by user. For example: SourceId_TargetId_1"],
        relationship_data: Annotated[dict, """Properties of the new relationship. The dictionary represent relationship to be sent to the topology.
            The format should be in Azure Digital Twins Definition Language (DTDL).
            It must follow the format:
            {
                "$relationshipId": "relation_id",
                "$sourceId": "source_id",
                "$relationshipName": "connects",
                "$targetId": "target_id",
                "$metadata": {
                    "$model": "model_id"
                }
            }

            where:
            - relation_id - the same unique identifier of the relationship as provided in relation_id argument
            - source_id  - the same unique identifier of the digital twin as provided in twin_id argument
            - "connects" - fixed keyword describes relationship
            - target_id - the unique identifier of the digital twin that is originally was created. This is id of twin that user ask to create or modify.
            - model_id - the unique identifier of the model that is used as source of relationship. Is obtained from model definition for source.
        """],
    ) -> Annotated[str, "Result of the operation. If operation is successful, the tool returns 'OK'. Otherwise, it returns 'FAIL' and the reason of failure, you're welcome to analyse error and fix it"]:
        try:
            self.client.upsert_relationship(twin_id, relationship_id, relationship_data)
            return "OK"
        except Exception as e:
            print("Error creating digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
        
    @kernel_function(
        name="delete_relationship",
        description="Deletes relationship between two twins in Azure Digital Twins Service."
    )
    def delete_relationship(
        self,
        twin_id: Annotated[str, """Unique identifier of the digital twin that is going to be the source of the connection.
            The identifier is a value of $dtId requested from Azure Digital Twins Query API"""],
        relationship_id: Annotated[str, """Unique identifier of the relationship that is going to be deleted.
            The identifier is a value of $relationshipId requested from Azure Digital Twins Query API
            For example: SourceId_TargetId_1"""],
    ) -> Annotated[str, "Result of the operation. If operation is successful, the tool returns 'OK'. Otherwise, it returns 'FAIL' and the reason of failure, you're welcome to analyse error and fix it"]:
        try:
            self.client.delete_relationship(twin_id, relationship_id)
            return "OK"
        except Exception as e:
            print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'

    @kernel_function(
        name="query_service", 
        description="""Request information about twins or relationships in Azure Digital Twins Service.
            This function should be used whan you need to get information about particular twin or several items or relationships between them."""
        )
    def query_service(
        self,
        query: Annotated[str, """Query in Azure Digital Twins Query API format that is going to be executed on Azure Digital Twin Service.
         
        For example: You need to know the details of ControlHub twin.
        From topology you may get model definition for ControlHub iand from definition understand that id is dtmi:com:example:ControlHub;1
        Then you can format request as:
        SELECT * FROM digitaltwins T WHERE IS_OF_MODEL(T, 'dtmi:com:example:ControlHub;1')

        The request returns all description of all twins with type ControlHub."""],
    ):
        if (query == None):
            return "FAIL: Query is not defined!"

        try:
            query_result = self.client.query_twins(query)
            json_result = json.dumps(list(query_result))
            return json_result
        except Exception as e:
            print("Error deleting digital twin in Azure Digital Twins service topology:", str(e))
            return f'FAIL: + {str(e)}\n'
        
# Add plugin to the kernel
kernel.add_plugin(plugin=AdtServicePlugin(), plugin_name="adt_servcie")

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing decorator for function: get_model_definition
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing param: model_type
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing annotation: typing.Annotated[str, 'Model type that definition is required. For example: ControlHub or TermoSensor']
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing param: model_type
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing annotation: <class 'str'>
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing param: model_type
DEBUG:semantic_kernel.functions.kernel_function_decorator:Parsing annotation: <class 'str'>
DEBUG:semantic_kernel.functions.kernel_function_decorator:annotations=[{'name': 'model_type', 'is_re

KernelPlugin(name='adt_servcie', description=None, functions={'create_twin': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='create_twin', plugin_name='adt_servcie', description='Creates (adds) new digital twin in Azure Digital Twins Service.', parameters=[KernelParameterMetadata(name='twin_id', description='Unique identifier of the new digital twin. For example: ControlHub_1', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'Unique identifier of the new digital twin. For example: ControlHub_1'}, function_schema_include=True), KernelParameterMetadata(name='properties', description='Properties of the new digital twin. The dictionary represent digital tween to be sent to the topology.\n            The format should be in Azure Digital Twins Definition Language (DTDL).\n            IMPORTANT! The metadata section should contain id of the model, for example if\n            model id is dtmi:com:exampl

### Prompt engineering

Define _system prompt_ that setup the stage.

In [4]:
system_message = """
    I am a user who is working on support digital twins in Azure Digital Twins Service. I will need you support to perform operation that I will request.
    
    The service defines several models (ontology) as follow:
    - Model CentralUnit - which is main unit of the system exists in one instance. 
        You cannot create CentralUnit as it is already defined. If the creation will be requested refuse it.
    - Model ControlHub - which is connection unit between CentralUnit and TermoSensors.
    - Model TermoSensor - which conects to ControlHub and reports temperature value.

    If user wants to create, update, delte or request information about any unit you must use appropriate tool(s) and provide valid parameters. 
    Digital Twin Definition Language (DTDL) must be used to communicate with Azure Digital Twins Service. Note that properties are case sensetive
    and must be passed exactly as defined in model, ignore the case of properties in user's input.

    If user requests to create a new digital twin you must also specify relationship between this twin that acts as target and the other that acts as source.
    You should never create twin without connection to source.

    If user wants update or delete exsted twin then user must specify twin id. If id is not provided, ask about it.

    Relationship between digital twins are stored separately from twins. If user request information where connection between twins presens you must request
    relationship information from relationships collection!
    For example for question: what temperature sensor connected to hub with id Node_23 valid query will be:
    SELECT * FROM relationships WHERE $sourceId = 'Node_23'
    Note request was done for relationships not digitaltwins collection.


    If you requested to create several twins (batch) you should think as follows:
        - identify if there is hierarchy between twins, for example is there requested ControlHub and TermoSensors.
        - if hierarchy presents you should start from "higher" twin (ControlHub) and create it accordingly to defined rules, when creation is done continue with
        "lower" twins (TermoSensors) and create them accordingly to defined rules.
        - if hierarchy is not presents you can create twins in any order.
        - once all twins are created you can create relationships between them as requested.
        - ControlHub must be connected to CentralUnit, TermoSensors must be connected to ControlHub.

    The request is following:
    
    """


### Query

Is used ReAct approach that let LLM to think what steps should be done to achieve task and what tool is better fit this needs.
Several agent were cinsidered from [available](https://github.com/microsoft/semantic-kernel/blob/main/python/samples/getting_started/05-using-the-planner.ipynb)

Set up the planner

In [ ]:
from semantic_kernel.planners.function_calling_stepwise_planner import (
    FunctionCallingStepwisePlanner,
    FunctionCallingStepwisePlannerOptions,
)

options = FunctionCallingStepwisePlannerOptions(
    max_iterations=10,
    max_tokens=4000
)

planner = FunctionCallingStepwisePlanner(service_id=service_id, options=options)

#### Common query

In [7]:
request = "How many TermoSensor do we have in out topology?"

#### ControlHub
- Add

In [11]:
request="Please create Control Hub located in Dnipro with status: on."

- Delete

In [17]:
request="Please remove ControlHub_Dnipro1 from topology."

#### Batch Operations

Request to add/remove to topology several twins as one user's request

In [10]:
request="""I want to add new location that contains 4 sensors. Status for all of them is off. 
    They should be conected to ControlHub that is located in Lviv give the hub id meaningful identifier that reflect its location.
    Location of sensors are the same as the hub.
"""

- Remove several twins

In [13]:
request="""It seems that ControlHub - ControlHub_Lviv is not working well.
Please remove it and all TermoSensors that are connected to this hub."""

### Demo requests

In [25]:
request="Please create Control Hub located in Dnipro with status: on."

In [28]:
request="Please create Termosensor located in Dnipro with status: on temperature 25 and connected to ControlHub located in Dnipro"

In [31]:
request="""I want to add new location that contains 4 sensors. Status for all of them is off. 
    They should be conected to ControlHub that is located in Lviv give the hub id meaningful identifier that reflect its location.
    Location of sensors are the same as the hub.
"""

### Execution

Debug tips:
 - function call starts with `INFO:semantic_kernel.kernel:Calling`

In [ ]:
question = system_message + request

result = await planner.invoke(kernel, question)

In [33]:
print(result.final_answer)

New location with a ControlHub in Lviv and 4 TermoSensors has been successfully added. All devices are currently off and correctly connected.
